# Object_Detection_YOLOv5 (Training)
1. 需有前處理的CSV和標籤集跟圖檔集。
1. 修改完設定檔後，執行前需修改[訓練模型](#5.4)指令。

<a class="anchor" id="0"></a>
# Table of Contents

1. [套件安裝與載入](#1)
1. [環境檢測與設定](#2)
1. [開發參數設定](#3)
1. [資料處理](#4)
    -  [載入CSV檔](#4.1)
    -  [檢查CSV檔缺失值](#4.2)
1. [製作資料＆訓練模型](#5)
    -  [製作資料集與標籤集](#5.1)
    -  [Get Class Name](#5.2)
    -  [製作訓練檔案路徑](#5.3)
    -  [訓練模型](#5.4)
    -  [Class Distribution](#5.5)
    -  [Batch ImageBatch Image](#5.6)
    -  [GT Vs PredGT Vs Pred](#5.7)
    -  [(Loss, Map) Vs Epoch(Loss, Map) Vs Epoch](#5.8)
    -  [Confusion Matrix](#5.9)
1. [待辦事項](#6)

# 1. 套件安裝與載入<a class="anchor" id="1"></a>
[Back to Table of Contents](#0)

In [ ]:
# 資料處理套件
import os
import gc
import yaml
import time
import shutil
import datetime
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from tqdm import tqdm
from sklearn.model_selection import train_test_split, GroupKFold

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# 設定顯示中文字體
from matplotlib.font_manager import FontProperties
plt.rcParams['font.sans-serif'] = ['Microsoft JhengHei'] # 用來正常顯示中文標籤
plt.rcParams['font.family'] = 'AR PL UMing CN'
plt.rcParams['axes.unicode_minus'] = False # 用來正常顯示負號

In [ ]:
# pytorch深度學習模組套件
import torch

# 2. 環境檢測與設定<a class="anchor" id="2"></a>
[Back to Table of Contents](#0)

In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DEVICE

In [ ]:
# 查看pytorch版本
print(torch.__version__)

In [ ]:
'''執行環境參數設定'''

# (Boolean)是否為本機
LOCAL = False

# (Boolean)是否為 Colab
COLAB = False


'''檔案路徑參數設定'''

# (String)Root路徑
if LOCAL:
    PATH = r'../'
elif COLAB:
    PATH = r'/content/drive/My Drive/Colab Notebooks/'
else:
    PATH = r'../input/'
    OUTPUT_PATH = r'/kaggle/working/'
    
# (String)資料根路徑
DATA_ROOT_PATH = PATH+r'vinbigdata-256-image-dataset/' 

# (String)標籤根路徑
LABEL_ROOT_PATH = PATH+r'labeldataset/' 

# (String)CSV根路徑
CSV_ROOT_PATH = PATH+r'trainyolo/' 

# (String)訓練資料路徑
TRAIN_DATA_PATH = DATA_ROOT_PATH+r'vinbigdata/train/'

# (String)訓練標籤路徑
LABEL_DATA_PATH = LABEL_ROOT_PATH+r'labels/'

# (String)訓練CSV路徑
TRAIN_CSV_PATH = CSV_ROOT_PATH+r'train_yolo.csv'

# (String)專案名稱
PROJECT_NAME = 'vinbigdata-chest-xray-abnormalities-detection'

# (Boolean)是否要匯入Library
IMPORT_PYTORCH_LIBRARY = False

# (String)Library的路徑
PYTORCH_LIBRARY_PATH = PATH + "PyTorch_Library/"

# (String)專案檔案儲存路徑
if LOCAL or COLAB:
    OUTPUT_PATH = PATH
PROJECT_PATH = OUTPUT_PATH+PROJECT_NAME+'/'+PROJECT_NAME+' '+datetime.datetime.now().strftime("%Y-%m-%d %H:%M")

# (String)權重名稱(使用哪個權重)
WEIGHTS_NAME = 'efficientnet_b0'

# (String)模型名稱(使用哪個模型)
MODEL_NAME = 'efficientnet_b0'

# (String)讀取預訓練權重的儲存路徑
LOAD_WEIGHTS_PATH = PROJECT_PATH+r'/models/pretrain_weights/'+WEIGHTS_NAME+'.pth'

# (String)讀取預訓練模型的儲存路徑
LOAD_MODEL_PATH = PROJECT_PATH+r'/models/pretrain_models/'+MODEL_NAME+'.pth'

# (Boolean)是否建立訓練模型儲存的時間戳資料夾
MODEL_TIME_FOLDER = False
# (String)訓練模型的儲存路徑
if MODEL_TIME_FOLDER:
    TRAIN_MODEL_PATH = PROJECT_PATH+r'/models/'+MODEL_NAME+'.pth'
else:
    TRAIN_MODEL_PATH = MODEL_NAME+'.pth'

In [ ]:
if DEVICE != torch.device("cpu"):
    !nvidia-smi

In [ ]:
if DEVICE != torch.device("cpu"):
    !nvcc --version

In [ ]:
if not LOCAL and COLAB:
    from google.colab import drive
    drive.mount('/content/drive')

if MODEL_TIME_FOLDER:
    if not os.path.isdir(PROJECT_PATH+r'/models/'):
        os.makedirs(PROJECT_PATH+r'/models/')
    
if IMPORT_PYTORCH_LIBRARY:
    sys.path.append(PYTORCH_LIBRARY_PATH + "Custom_Loss.py")
    sys.path.append(PYTORCH_LIBRARY_PATH + "Custom_Model.py")

# 3. 開發參數設定<a class="anchor" id="3"></a>
[Back to Table of Contents](#0)

In [ ]:
'''客製參數設定'''


'''資料參數設定'''

# (Int)有CSV檔該參數才有用，1則為不做交叉驗證
FOLD = 5

# (String)圖片副檔名
IMAGE_NAME_EXTENSION = '.png'

# (String)CSV圖片檔名欄位
IMAGE_NAME = 'image_id'

# (String)CSV標籤欄位
LABEL_NAME = 'class_name'

# (String)CSV標籤ID欄位
LABEL_ID = 'class_id'

# (Boolean)是否有空物件框的csv資料
EMPTY_BOUNDING_BOX = True

if EMPTY_BOUNDING_BOX:
    # (Int)CSV空物件框標籤ID
    EMPTY_BOUNDING_BOX_LABEL_ID = 14
    
# (Int)不同的種子會產生不同的Random或分層K-FOLD分裂, 42則是預設固定種子
SEED = 42

if FOLD == 1:
    # (Float)驗證集佔訓練集的比率，FOLD>1則不啟用
    DATA_SPLIT = 0.2
else:
    # (String)切分訓練集跟驗證集方式 GroupKFold
    KF = GroupKFold(n_splits = FOLD)


''''圖表參數設定'''

# (Float)全部SNS圖表的字形縮放
ALL_SNS_FONT_SCALE = 1.0

# (Int)CSV缺失值圖表寬度
CSV_COUNTPLOT_FIGSIZE_W = 10

# (Int)CSV缺失值圖表高度
CSV_COUNTPLOT_FIGSIZE_H = 10

# (Int)CSV缺失值圖表標題字型大小
CSV_COUNTPLOT_TITLE_FONTSIZE = 20

# (Int)CSV缺失值圖表X軸標題字型大小
CSV_COUNTPLOT_XLABEL_FONTSIZE = 15

# (Int)CSV缺失值圖表Y軸標題字型大小
CSV_COUNTPLOT_YLABEL_FONTSIZE = 15

# (Int)Class Distribution圖表寬度
CLASS_DISTRIBUTION_FIGSIZE_W = 20

# (Int)Class Distribution圖表高度
CLASS_DISTRIBUTION_FIGSIZE_H = 20

# (Int)Batch Image圖表寬度
BATCH_IMAGE_FIGSIZE_W = 15

# (Int)Batch Image圖表高度
BATCH_IMAGE_FIGSIZE_H = 15

# (Int)Batch Image顯示次數
COUNTS = 3

# (Int)GT Vs Pred圖表figsize倍率
FIGSIZE_RATE = 5

# (Boolean)GT Vs Pred圖表是否自動調整子圖間距
CONSTRAINED_LAYOUT = True

# (Int)GT Vs Pred圖表標題字體大小
FONTSIZE = 12

# (Int)(Loss, Map) Vs Epoch圖表寬度
LOSS_MAP_EPOCH_FIGSIZE_W = 30

# (Int)(Loss, Map) Vs Epoch圖表高度
LOSS_MAP_EPOCH_FIGSIZE_H = 15

# (Int)Confusion Matrix圖表寬度
CONFUSION_MATRIX_FIGSIZE_W = 30

# (Int)Confusion Matrix圖表高度
CONFUSION_MATRIX_FIGSIZE_H = 15

In [ ]:
# 設置sns圖表縮放係數
sns.set(font_scale = ALL_SNS_FONT_SCALE)

# 4. 資料處理<a class="anchor" id="4"></a>
[Back to Table of Contents](#0)

## 4.1 載入CSV檔 <a class="anchor" id="4.1"></a>
[Back to Table of Contents](#0)

In [ ]:
print('Reading data...')

# 讀取訓練資料集CSV檔
train_csv = pd.read_csv(TRAIN_CSV_PATH,encoding="utf8")
if EMPTY_BOUNDING_BOX:
    train_csv = train_csv[train_csv.class_id != EMPTY_BOUNDING_BOX_LABEL_ID].reset_index(drop = True)

print('Reading data completed')

In [ ]:
# 顯示訓練資料集CSV檔
train_csv.head()

In [ ]:
print("Shape of train_data :", train_csv.shape)

## 4.2 檢查CSV檔缺失值 <a class="anchor" id="4.2"></a>
[Back to Table of Contents](#0)

In [ ]:
# 缺失值比率
total = train_csv.isnull().sum().sort_values(ascending = False)
percent = (train_csv.isnull().sum()/train_csv.isnull().count()*100).sort_values(ascending = False)
missing_train_csv  = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_train_csv.head(missing_train_csv.shape[0])

In [ ]:
train_csv[LABEL_NAME].value_counts()

In [ ]:
f,ax = plt.subplots(figsize=(CSV_COUNTPLOT_FIGSIZE_W, CSV_COUNTPLOT_FIGSIZE_H))
sns.countplot(train_csv[LABEL_NAME], hue = train_csv[LABEL_NAME],ax = ax)
plt.title("LABEL COUNT", fontsize=CSV_COUNTPLOT_TITLE_FONTSIZE)
plt.xlabel(LABEL_NAME.upper(), fontsize=CSV_COUNTPLOT_XLABEL_FONTSIZE)
plt.ylabel("COUNT", fontsize=CSV_COUNTPLOT_YLABEL_FONTSIZE)
plt.legend()
plt.show()

# 5. 製作資料＆訓練模型 <a class="anchor" id="5"></a>
[Back to Table of Contents](#0)

## 5.1 製作資料集與標籤集 <a class="anchor" id="5.1"></a>
[Back to Table of Contents](#0)

In [ ]:
os.makedirs(OUTPUT_PATH+'train/labels/', exist_ok = True)
os.makedirs(OUTPUT_PATH+'train/images/', exist_ok = True)
files = []
files = train_csv['image_path'].unique()
for file in tqdm(files):
    shutil.copy(file, OUTPUT_PATH+'train/images/')
    filename = file.split('/')[-1].split('.')[0]
    shutil.copy(os.path.join(LABEL_DATA_PATH, filename+'.txt'), OUTPUT_PATH+'train/labels/')

## 5.2 Get Class Name <a class="anchor" id="5.2"></a>
[Back to Table of Contents](#0)

In [ ]:
class_ids, class_names = list(zip(*set(zip(train_csv[LABEL_ID], train_csv[LABEL_NAME]))))
classes = list(np.array(class_names)[np.argsort(class_ids)])
classes = list(map(lambda x: str(x), classes))
classes

## 5.3 製作訓練檔案路徑 <a class="anchor" id="5.3"></a>
[Back to Table of Contents](#0)

In [ ]:
def txt_process(fold, kf): 
    train_files = []
    val_files = []
    val_files += list(train_csv[train_csv.Folds == fold].image_path.unique())
    train_files += list(train_csv[train_csv.Folds != fold].image_path.unique())
    print(len(train_files), len(val_files))
    
    os.makedirs(OUTPUT_PATH+'train/'+str(fold)+'/', exist_ok = True)

    with open(os.path.join(OUTPUT_PATH, 'train/'+str(fold)+'/train.txt'), 'w') as f:
        for path in train_files:
            filename = path.split('/')[-1].split('.')[0]
            f.write(OUTPUT_PATH+'train/images/'+filename+IMAGE_NAME_EXTENSION+'\n')

    with open(os.path.join(OUTPUT_PATH, 'train/'+str(fold)+'/val.txt'), 'w') as f:
        for path in val_files:
            filename = path.split('/')[-1].split('.')[0]
            f.write(OUTPUT_PATH+'train/images/'+filename+IMAGE_NAME_EXTENSION+'\n')

    data = dict(
        train =  os.path.join(OUTPUT_PATH, 'train/'+str(fold)+'/train.txt') ,
        val   =  os.path.join(OUTPUT_PATH, 'train/'+str(fold)+'/val.txt' ),
        nc    = len(classes),
        names = classes
        )

    with open(os.path.join(OUTPUT_PATH, 'train/'+str(fold)+'/train.yaml'), 'w') as outfile:
        yaml.dump(data, outfile, default_flow_style=False)

    f = open(os.path.join(OUTPUT_PATH, 'train/'+str(fold)+'/train.yaml'), 'r')
    print('\nyaml:')
    print(f.read())

In [ ]:
def txt_main():
    try:
        if FOLD > 1:
            train_csv['Folds'] = -1
            for fold, (train_index, valid_index) in enumerate(KF.split(train_csv, groups = train_csv[IMAGE_NAME].tolist())):
                train_csv.loc[valid_index, 'Folds'] = fold
                txt_process(fold = fold, kf = True)
        else:
            txt_process(fold = 0, kf = False)
    except Exception as exception:
        print(exception)
        raise

In [ ]:
if __name__ == '__main__':
    txt_main()

## 5.4 訓練模型 <a class="anchor" id="5.4"></a>
[Back to Table of Contents](#0)

In [ ]:
%cd train
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
%pip install -qr requirements.txt  # install dependencies

In [ ]:
!WANDB_MODE="dryrun" python train.py --img 256 --batch 32 --epochs 3 --data /kaggle/working/train/0/train.yaml --weights yolov5x.pt --cache

## 5.5 Class Distribution <a class="anchor" id="5.5"></a>
[Back to Table of Contents](#0)

In [ ]:
plt.figure(figsize = (CLASS_DISTRIBUTION_FIGSIZE_W,CLASS_DISTRIBUTION_FIGSIZE_H))
plt.axis('off')
plt.imshow(plt.imread('runs/train/exp/labels_correlogram.jpg'));

In [ ]:
plt.figure(figsize = (CLASS_DISTRIBUTION_FIGSIZE_W,CLASS_DISTRIBUTION_FIGSIZE_H))
plt.axis('off')
plt.imshow(plt.imread('runs/train/exp/labels.jpg'));

## 5.6 Batch ImageBatch Image <a class="anchor" id="5.6"></a>
[Back to Table of Contents](#0)

In [ ]:
for i in range(COUNTS):
    plt.figure(figsize = (BATCH_IMAGE_FIGSIZE_W, BATCH_IMAGE_FIGSIZE_H))
    plt.imshow(plt.imread('runs/train/exp/train_batch'+str(i)+'.jpg'))

## 5.7 GT Vs PredGT Vs Pred <a class="anchor" id="5.7"></a>
[Back to Table of Contents](#0)

In [ ]:
fig, ax = plt.subplots(COUNTS, 2, figsize = (2*FIGSIZE_RATE,COUNTS*FIGSIZE_RATE), constrained_layout = CONSTRAINED_LAYOUT)
for row in range(COUNTS):
    ax[row][0].imshow(plt.imread(f'runs/train/exp/test_batch{row}_labels.jpg'))
    ax[row][0].set_xticks([])
    ax[row][0].set_yticks([])
    ax[row][0].set_title(f'runs/train/exp/test_batch{row}_labels.jpg', fontsize = FONTSIZE)
    
    ax[row][1].imshow(plt.imread(f'runs/train/exp/test_batch{row}_pred.jpg'))
    ax[row][1].set_xticks([])
    ax[row][1].set_yticks([])
    ax[row][1].set_title(f'runs/train/exp/test_batch{row}_pred.jpg', fontsize = FONTSIZE)

## 5.8 (Loss, Map) Vs Epoch <a class="anchor" id="5.8"></a>
[Back to Table of Contents](#0)

In [ ]:
plt.figure(figsize=(LOSS_MAP_EPOCH_FIGSIZE_W,LOSS_MAP_EPOCH_FIGSIZE_H))
plt.axis('off')
plt.imshow(plt.imread('runs/train/exp/results.png'));

## 5.9 Confusion Matrix <a class="anchor" id="5.9"></a>
[Back to Table of Contents](#0)

In [ ]:
plt.figure(figsize=(CONFUSION_MATRIX_FIGSIZE_W,CONFUSION_MATRIX_FIGSIZE_H))
plt.axis('off')
plt.imshow(plt.imread('runs/train/exp/confusion_matrix.png'));

# 6. 待辦事項<a class="anchor" id="6"></a>
[Back to Table of Contents](#0)

[Go to Top](#0)